# MEMS Spring-Mass-Damper System Simulation

**Author:** Silicon Fabrication Handbook  
**License:** MIT  
**Description:** Interactive simulation of a MEMS accelerometer modeled as a spring-mass-damper system

---

## Table of Contents

1. [Introduction](#introduction)
2. [Theory](#theory)
3. [Setup and Imports](#setup)
4. [MEMS Accelerometer Class](#class)
5. [Step Response Analysis](#step-response)
6. [Frequency Response (Bode Plot)](#frequency-response)
7. [Noise Analysis](#noise-analysis)
8. [Transient Response](#transient-response)
9. [Design Space Exploration](#design-space)
10. [Interactive Parameter Tuning](#interactive)
11. [Summary and Conclusions](#summary)

## 1. Introduction <a id="introduction"></a>

MEMS (Micro-Electro-Mechanical Systems) accelerometers are widely used in:
- Smartphones and tablets (screen rotation, step counting)
- Automotive safety (airbag deployment, ESC)
- Consumer electronics (gaming controllers, wearables)
- Industrial applications (vibration monitoring)

This notebook simulates a **capacitive MEMS accelerometer** using a **spring-mass-damper** model.

### Key Features:
- ✅ Time-domain response analysis
- ✅ Frequency response (Bode plots)
- ✅ Noise analysis (thermal + electronic)
- ✅ Capacitive sensing simulation
- ✅ Design trade-off exploration
- ✅ Interactive parameter tuning

## 2. Theory <a id="theory"></a>

### 2.1 Equation of Motion

The proof mass motion is governed by:

$$
m\ddot{x} + b\dot{x} + kx = ma(t)
$$

Where:
- $m$ = proof mass [kg]
- $b$ = damping coefficient [N·s/m]
- $k$ = spring constant [N/m]
- $x$ = displacement [m]
- $a(t)$ = input acceleration [m/s²]

### 2.2 Key Parameters

**Natural frequency:**
$$\omega_n = \sqrt{\frac{k}{m}} \quad [rad/s]$$

**Damping ratio:**
$$\zeta = \frac{b}{2\sqrt{mk}}$$

**Quality factor:**
$$Q = \frac{1}{2\zeta}$$

### 2.3 Transfer Function

In Laplace domain:
$$H(s) = \frac{X(s)}{A(s)} = \frac{1}{s^2 + 2\zeta\omega_n s + \omega_n^2}$$

### 2.4 Capacitive Sensing

Parallel plate capacitance:
$$C = \frac{\epsilon_0 A}{d_0 - x}$$

For small displacement:
$$\Delta C \approx C_0 \frac{x}{d_0}$$

## 3. Setup and Imports <a id="setup"></a>

In [ ]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# For interactive widgets
try:
    from ipywidgets import interact, FloatSlider, IntSlider, Output
    import ipywidgets as widgets
    WIDGETS_AVAILABLE = True
except ImportError:
    print("⚠️  ipywidgets not installed. Interactive features disabled.")
    print("   Install with: pip install ipywidgets")
    WIDGETS_AVAILABLE = False

# Configure plotting
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['lines.linewidth'] = 2

# Create output directory
OUTPUT_DIR = Path("images")
OUTPUT_DIR.mkdir(exist_ok=True)

print("✅ Setup complete!")
print(f"  NumPy version: {np.__version__}")
print(f"  Output directory: {OUTPUT_DIR}")
print(f"  Interactive widgets: {'Enabled' if WIDGETS_AVAILABLE else 'Disabled'}")

## 4. MEMS Accelerometer Class <a id="class"></a>

In [ ]:
class MEMSAccelerometer:
    """
    MEMS accelerometer spring-mass-damper model.
    
    Parameters:
        m: Proof mass [kg]
        k: Spring constant [N/m]
        b: Damping coefficient [N·s/m]
        C0: Nominal capacitance [F]
        d0: Nominal gap [m]
        A: Electrode area [m²]
    """
    
    def __init__(self, m=1e-9, k=10, b=1e-6, C0=1e-12, d0=2e-6, A=1e-8):
        self.m = m
        self.k = k
        self.b = b
        self.C0 = C0
        self.d0 = d0
        self.A = A
        
        # Derived parameters
        self.omega_n = np.sqrt(k / m)
        self.f_n = self.omega_n / (2 * np.pi)
        self.zeta = b / (2 * np.sqrt(m * k))
        self.Q = 1 / (2 * self.zeta) if self.zeta > 0 else np.inf
        self.sensitivity = 1 / (k / m)
    
    def __repr__(self):
        return f"""MEMS Accelerometer:
  Mass: {self.m*1e9:.2f} ng
  Spring constant: {self.k:.2f} N/m
  Damping: {self.b*1e6:.3f} µN·s/m
  Natural frequency: {self.f_n/1e3:.2f} kHz
  Damping ratio ζ: {self.zeta:.3f}
  Quality factor Q: {self.Q:.1f}
  Sensitivity: {self.sensitivity*1e9:.2f} nm/g"""
    
    def transfer_function(self):
        """Return H(s) = X(s)/A(s)"""
        num = [1]
        den = [1, 2*self.zeta*self.omega_n, self.omega_n**2]
        return signal.TransferFunction(num, den)
    
    def step_response(self, acceleration=9.81, t_max=0.01, n_points=1000):
        """Simulate step response."""
        sys = self.transfer_function()
        t = np.linspace(0, t_max, n_points)
        t_step, x = signal.step(sys, T=t)
        return t_step, x * acceleration
    "
    def frequency_response(self, f_min=1, f_max=1e6, n_points=1000):
        """Calculate frequency response."""
        sys = self.transfer_function()
        w = 2 * np.pi * np.logspace(np.log10(f_min), np.log10(f_max), n_points)
        w, H = signal.freqs(sys.num, sys.den, worN=w)
        return w / (2 * np.pi), np.abs(H)
    "
    def displacement_to_capacitance(self, x):
        """Convert displacement to capacitance."""
        epsilon_0 = 8.854e-12
        return epsilon_0 * self.A / (self.d0 - x)
    "
    def capacitance_to_voltage(self, C, V_bias=1.0, C_f=1e-12):
        """Convert capacitance to voltage."""
        delta_C = C - self.C0
        return -V_bias * delta_C / C_f

# Create default instance
accel = MEMSAccelerometer()
print(accel)